In [12]:
import duckdb
from shapely.geometry import Point
import geopandas as gpd
from pyproj import CRS
# from keplergl import KeplerGl
import pandas as pd

In [13]:
duckdb_folder = '../duckdb'
data_folder = '../data'

con = duckdb.connect(duckdb_folder+'/kepler.duckdb')

In [14]:
data_raw = con.execute('SELECT * FROM "trips"').df()
con.close()

In [15]:
data=data_raw[:20000]

In [16]:
data['geometry'] = [Point(long, lat) for long, lat in zip(data['posx'].to_list(), data['posy'].to_list())]
geodata = gpd.GeoDataFrame(data, crs = CRS.from_epsg('4326'))
geodata

/tmp/ipykernel_65136/2678564112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['geometry'] = [Point(long, lat) for long, lat in zip(data['posx'].to_list(), data['posy'].to_list())]


,tripid,vehicleid,posx,posy,t,geometry
0,1,1,4.457924,50.889049,2020-06-01 11:48:50.886000+02:00,POINT (4.45792 50.88905)
1,1,1,4.457894,50.889028,2020-06-01 11:48:52.386000+02:00,POINT (4.45789 50.88903)
2,1,1,4.457721,50.888909,2020-06-01 11:48:57.523414+02:00,POINT (4.45772 50.88891)
3,1,1,4.457928,50.888690,2020-06-01 11:49:05.623414+02:00,POINT (4.45793 50.88869)
4,1,1,4.457930,50.888689,2020-06-01 11:49:05.668849+02:00,POINT (4.45793 50.88869)
...,...,...,...,...,...,...
19995,10,1,4.410637,50.801712,2020-06-05 00:14:04.531535+02:00,POINT (4.41064 50.80171)
19996,10,1,4.410675,50.801728,2020-06-05 00:14:05.131535+02:00,POINT (4.41068 50.80173)
19997,10,1,4.410713,50.801743,2020-06-05 00:14:06.131535+02:00,POINT (4.41071 50.80174)
19998,10,1,4.411239,50.801961,2020-06-05 00:14:14.531535+02:00,POINT (4.41124 50.80196)


In [17]:
# Create timestamp with Pandas datetime. Inclue None as timezone
geodata['time'] = pd.to_datetime(geodata['t']).dt.tz_localize(None)
# Set timestamp as Index
geodata = geodata.set_index('time')
print(geodata.shape)
geodata.head()


(20000, 6)


,tripid,vehicleid,posx,posy,t,geometry
time,,,,,,
2020-06-01 11:48:50.886000,1,1,4.457924,50.889049,2020-06-01 11:48:50.886000+02:00,POINT (4.45792 50.88905)
2020-06-01 11:48:52.386000,1,1,4.457894,50.889028,2020-06-01 11:48:52.386000+02:00,POINT (4.45789 50.88903)
2020-06-01 11:48:57.523414,1,1,4.457721,50.888909,2020-06-01 11:48:57.523414+02:00,POINT (4.45772 50.88891)
2020-06-01 11:49:05.623414,1,1,4.457928,50.888690,2020-06-01 11:49:05.623414+02:00,POINT (4.45793 50.88869)
2020-06-01 11:49:05.668849,1,1,4.457930,50.888689,2020-06-01 11:49:05.668849+02:00,POINT (4.45793 50.88869)


In [18]:
import movingpandas as mpd
traj_collection = mpd.TrajectoryCollection(geodata, 'tripid')

In [19]:
traj_collection

TrajectoryCollection with 10 trajectories

In [20]:
from datetime import datetime, timedelta
SearchRadio = 100
# stop detection
stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(seconds=1), max_diameter=SearchRadio)

In [21]:
stops

TrajectoryCollection with 1067 trajectories

In [22]:
stops_start = gpd.GeoDataFrame(columns = ['geometry'])
stops_start = stops_start.set_geometry('geometry')

In [23]:
stops_start['stop_id'] = [track.id for track in stops.trajectories]
stops_start= stops_start.set_index('stop_id')


In [24]:
for stoptrack in stops.trajectories:

    # add stop duration in hours
    stops_start.at[stoptrack.id,'duration_h'] =stoptrack.get_duration().total_seconds()/3600

    # add length
    stops_start.at[stoptrack.id, 'length_m']=stoptrack.get_length()

    # add bird name
    stops_start.at[stoptrack.id, 'tripid']=stoptrack.id.split('_')[0]

    # add datetime
    stops_start.at[stoptrack.id, 'datetime']= pd.to_datetime(stoptrack.id.split('_')[1]).tz_localize(None)

    # geometry with start point
    stops_start.at[stoptrack.id, 'geometry'] = stoptrack.get_start_location()
print(stops_start.shape)
stops_start.head()


(1067, 5)


,geometry,duration_h,length_m,tripid,datetime
stop_id,,,,,
1_2020-06-01 11:48:50.886000,POINT (4.45792 50.88905),0.008360,96.957224,1,2020-06-01 11:48:50.886000
1_2020-06-01 11:49:26.624003,POINT (4.45843 50.88818),0.009625,89.403874,1,2020-06-01 11:49:26.624003
1_2020-06-01 11:50:04.874237,POINT (4.45968 50.88763),0.009784,132.079704,1,2020-06-01 11:50:04.874237
1_2020-06-01 11:50:40.597012,POINT (4.46036 50.88704),0.004817,84.711824,1,2020-06-01 11:50:40.597012
1_2020-06-01 11:51:01.899785,POINT (4.4605 50.88603),0.007334,67.020078,1,2020-06-01 11:51:01.899785


In [25]:
min(stops_start['datetime'])

Timestamp('2020-06-01 11:48:50.886000')

In [26]:
# Reset indexes
stops_start = stops_start.reset_index(drop=True)
geodata= geodata.reset_index(drop=True)

In [27]:
from keplergl import KeplerGl

m = KeplerGl(height=600)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [28]:
# Add stop durations
m.add_data(stops_start, 'stops')
# Add gps records
m.add_data(geodata, 'trajectories')

In [29]:
print(geodata.shape)
print(geodata.head())
print(stops_start.shape)
print(stops_start.head())

(20000, 6)
   tripid  vehicleid      posx       posy                                t  \
0       1          1  4.457924  50.889049 2020-06-01 11:48:50.886000+02:00   
1       1          1  4.457894  50.889028 2020-06-01 11:48:52.386000+02:00   
2       1          1  4.457721  50.888909 2020-06-01 11:48:57.523414+02:00   
3       1          1  4.457928  50.888690 2020-06-01 11:49:05.623414+02:00   
4       1          1  4.457930  50.888689 2020-06-01 11:49:05.668849+02:00   

                   geometry  
0  POINT (4.45792 50.88905)  
1  POINT (4.45789 50.88903)  
2  POINT (4.45772 50.88891)  
3  POINT (4.45793 50.88869)  
4  POINT (4.45793 50.88869)  
(1067, 5)
                   geometry  duration_h    length_m tripid  \
0  POINT (4.45792 50.88905)    0.008360   96.957224      1   
1  POINT (4.45843 50.88818)    0.009625   89.403874      1   
2  POINT (4.45968 50.88763)    0.009784  132.079704      1   
3  POINT (4.46036 50.88704)    0.004817   84.711824      1   
4   POINT (4.4605 50

In [30]:
print(m.config)

{}


In [31]:
m

KeplerGl(data={'stops': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [32]:
# m.save_to_html(file_name='index.html', config=config, read_only=True)

In [33]:
print(m.config)

{}


In [34]:
# Check the actual column names in your data
print("Columns in geodata:", geodata.columns.tolist())
print("Columns in stops:", stops_start.columns.tolist())

Columns in geodata: ['tripid', 'vehicleid', 'posx', 'posy', 't', 'geometry']
Columns in stops: ['geometry', 'duration_h', 'length_m', 'tripid', 'datetime']


In [38]:
# Fixed time-based configuration for trip data
import pandas as pd
from keplergl import KeplerGl

# Step 1: First filter your data to only 3 minutes to avoid date issues
print("Original data shapes:")
print(f"Trajectories: {geodata.shape}")
print(f"Stops: {stops_start.shape}")

# Get the earliest timestamp from both datasets (handle timezone issues)
traj_start = pd.to_datetime(geodata['t']).dt.tz_localize(None).min()  # Remove timezone
stops_earliest = pd.to_datetime(stops_start['datetime']).dt.tz_localize(None).min()  # Remove timezone

print(f"Trajectory start: {traj_start}")
print(f"Stops start: {stops_earliest}")

overall_start = min(traj_start, stops_earliest)
end_time = overall_start + pd.Timedelta(hours=3)

print(f"\nFiltering data from {overall_start} to {end_time}")

# Filter both datasets to first 3 minutes (handle timezone-aware data)
geodata_filtered = geodata[pd.to_datetime(geodata['t']).dt.tz_localize(None) <= end_time].copy()
stops_filtered = stops_start[pd.to_datetime(stops_start['datetime']).dt.tz_localize(None) <= end_time].copy()

print(f"\nFiltered data shapes:")
print(f"Trajectories: {geodata_filtered.shape}")
print(f"Stops: {stops_filtered.shape}")

# Step 2: Convert timestamps for time filter (Kepler needs milliseconds)
geodata_filtered['timestamp_ms'] = pd.to_datetime(geodata_filtered['t']).astype('int64') // 10**6
stops_filtered['timestamp_ms'] = pd.to_datetime(stops_filtered['datetime']).astype('int64') // 10**6

# Get time range in milliseconds (what Kepler expects)
min_time_ms = int(min(geodata_filtered['timestamp_ms'].min(), stops_filtered['timestamp_ms'].min()))
max_time_ms = int(max(geodata_filtered['timestamp_ms'].max(), stops_filtered['timestamp_ms'].max()))

print(f"\nTime range (ms): {min_time_ms} to {max_time_ms}")
print(f"Date range: {pd.to_datetime(min_time_ms, unit='ms')} to {pd.to_datetime(max_time_ms, unit='ms')}")

# Step 3: Create the time-based configuration with correct timestamp values
time_config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [
                {
                    "dataId": ["trajectories"],
                    "id": "time_filter_trips",
                    "name": ["t"],
                    "type": "timeRange",
                    "value": [min_time_ms, max_time_ms],  # Use milliseconds
                    "enlarged": True,
                    "plotType": "histogram",
                    "animationWindow": "free",
                    "yAxis": None
                }
            ],
            "layers": [
                
                {
                    "id": "6riwuft", 
                    "type": "geojson",
                    "config": {
                        "dataId": "trajectories",
                        "label": "trajectories",
                        "color": [137, 218, 193],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {
                            "geojson": "geometry"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.6,  # Slightly reduced for better layering
                            "strokeOpacity": 0.8,
                            "thickness": 1.5,  # Increased thickness for visibility
                            "strokeColor": None,
                            "colorRange": {
                                "name": "ColorBrewer Set1-9",  # Better for categorical data
                                "type": "qualitative", 
                                "category": "ColorBrewer",
                                "colors": ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#ffff33", "#a65628", "#f781bf", "#999999"]
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber", 
                                "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]
                            },
                            "radius": 10,
                            "sizeRange": [0, 10],
                            "radiusRange": [0, 50],
                            "heightRange": [0, 500],
                            "elevationScale": 5,
                            "enableElevationZoomFactor": True,
                            "stroked": True,  # Enable stroke for better definition
                            "filled": True,
                            "enable3d": False,
                            "wireframe": False
                        },
                        "hidden": False,
                        "textLabel": [{
                            "field": None,
                            "color": [255, 255, 255],
                            "size": 18,
                            "offset": [0, 0],
                            "anchor": "start", 
                            "alignment": "center",
                            "outlineWidth": 0,
                            "outlineColor": [255, 0, 0, 255],
                            "background": False,
                            "backgroundColor": [0, 0, 200, 255]
                        }]
                    },
                    "visualChannels": {
                        "colorField": {
                            "name": "tripid",  # Color by trip ID to match trajectories to trips
                            "type": "integer"
                        },
                        "colorScale": "ordinal",
                        "strokeColorField": {
                            "name": "vehicleid",  # Stroke color by vehicle
                            "type": "integer"
                        },
                        "strokeColorScale": "ordinal",
                        "sizeField": None,
                        "sizeScale": "linear",
                        "heightField": None,
                        "heightScale": "linear",
                        "radiusField": None,
                        "radiusScale": "linear"
                    }
                }
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "stops": [
                            {"name": "tripid", "format": None},
                            {"name": "duration_h", "format": None},
                            {"name": "length_m", "format": None},
                            {"name": "datetime", "format": None}
                        ],
                        "trajectories": [
                            {"name": "tripid", "format": None},
                            {"name": "vehicleid", "format": None},
                            {"name": "t", "format": None},
                            {"name": "posx", "format": None},
                            {"name": "posy", "format": None}
                        ]
                    },
                    "enabled": True
                },
                "brush": {"enabled": False},
                "geocoder": {"enabled": True}
            },
            "layerBlending": "normal",
            "animationConfig": {
                "currentTime": None,
                "speed": 1
            }
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": 50.84510327056364,
            "longitude": 4.3973137,
            "pitch": 0,
            "zoom": 13,  # Closer zoom for detailed view
            "isSplit": False
        },
        "mapStyle": {
            "styleType": "dark-matter",
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True
            }
        }
    }
}

# Step 4: Create the map with filtered data and configuration
print("\nCreating Kepler.gl map...")
m = KeplerGl(height=600, config=time_config)

# Add the FILTERED data (this is crucial!)
m.add_data(stops_filtered, 'stops')
m.add_data(geodata_filtered, 'trajectories')

# Display the map
print("Map created successfully!")
m

Original data shapes:
Trajectories: (20000, 6)
Stops: (1067, 5)
Trajectory start: 2020-06-01 11:48:50.886000
Stops start: 2020-06-01 11:48:50.886000

Filtering data from 2020-06-01 11:48:50.886000 to 2020-06-01 14:48:50.886000

Filtered data shapes:
Trajectories: (2103, 6)
Stops: (119, 5)

Time range (ms): 1591004930 to 1591015177014
Date range: 1970-01-19 09:56:44.930000 to 2020-06-01 12:39:37.014000

Creating Kepler.gl map...
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map created successfully!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['trajectories'], 'id': 'time…

In [39]:
m.save_to_html(file_name='index.html', read_only=True)

Map saved to index.html!
